# 天气预报微调

## 加载1.8B Chat Int4模型
在原有chat模型上进行参数微调

In [1]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

2024-03-01 07:46:03,194 - modelscope - INFO - PyTorch version 2.1.2+cu121 Found.
2024-03-01 07:46:03,196 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-03-01 07:46:03,319 - modelscope - INFO - Loading done! Current index file version is 1.12.0, with md5 db1505a3ac0bd41b605104b5cde0c395 and a total number of 964 components indexed


In [2]:
model_dir = snapshot_download('qwen/Qwen-1_8B-Chat-Int4')

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

2024-03-01 07:46:04,558 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
CUDA extension not installed.
CUDA extension not installed.
Try importing flash-attention for faster inference...
/root/miniconda3/lib/python3.10/site-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [3]:
prompt = '{\'id\': \'identity_1\', \'conversations\': [{\'from\': \'user\', \'value\': \'\\n给定一句话：“你们1999年4月8日去金溪县玩吗？”，请你按步骤要求工作。\\n\\n步骤1：识别这句话中的城市和日期共2个信息\\n步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}\\n\\n请问，这个JSON字符串是：\\n\'}, {\'from\': \'assistant\', \'value\': \'{"city": "金溪县", "date": "1999-04-08"}\'}]}'


In [4]:
response, _ = model.chat(tokenizer
                         , prompt
                         , history=None
                        )
print(response)

这个JSON字符串是："{\"city\":\"金溪县\",\"date\":\"1999-04-08\"}"。


## 有效参数微调

In [5]:
# 城市数据
with open('city.txt','r',encoding='utf-8') as fp:
    city_list=fp.readlines()
    city_list=[line.strip().split(' ')[1] for line in city_list]

In [6]:
Q='青岛4月6日下雨么?'

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''

# 生成SFT微调数据

Qwen的SFT数据格式要求:
```
[
  {
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "你好"
      },
      {
        "from": "assistant",
        "value": "我是一个语言模型，我叫通义千问。"
      }
    ]
  }
]
```

In [7]:
import random
import json
import time 

Q_arr=[]
A_arr=[]

Q_list=[
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),

    ('你们{year}年{month}月{day}日去{city}玩吗？','%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？','%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？','%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？','%m-%d'),
]

# 生成一批"1月2号"、"1月2日"、"2023年1月2号", "2023年1月2日", "2023-02-02", "03-02"之类的话术, 教会它做日期转换
for i in range(1000):
    Q=Q_list[random.randint(0,len(Q_list)-1)]
    city=city_list[random.randint(0,len(city_list)-1)]
    year=random.randint(1990,2025)
    month=random.randint(1,12)
    day=random.randint(1,28)
    time_str='{}-{}-{}'.format(year,month,day)
    date_field=time.strftime(Q[1],time.strptime(time_str,'%Y-%m-%d'))
    Q=Q[0].format(city=city,year=year,month=month,day=day) # 问题
    A=json.dumps({'city':city,'date':date_field},ensure_ascii=False)  # 回答

    Q_arr.append(prompt_template%(Q,))
    A_arr.append(A)

import pandas as pd 

df=pd.DataFrame({'Prompt':Q_arr,'Completion':A_arr})
df.to_excel('train.xlsx')

In [8]:
def get_finetinu_data(idx, Q, A):
    data_tample = {
    "id": f"identity_{idx}",
    "conversations": [
    {
    "from": "user",
    "value": f"{Q}"
    },
    {
    "from": "assistant",
    "value": f"{A}"
    }
    ]
    }
    return data_tample

In [9]:
data_res = []
for idx, items in enumerate(df.to_dict(orient="records")):
    # print(idx, key)
    data = get_finetinu_data(idx, items["Prompt"], items["Completion"])
    data_res.append(data)
    # break

In [10]:
df

,Prompt,Completion
0,\n给定一句话：“1998年7月24号市辖区的天气”，请你按步骤要求工作。\n\n步骤1：识...,"{""city"": ""市辖区"", ""date"": ""1998-07-24""}"
1,\n给定一句话：“你们9月19号去江城区玩吗？”，请你按步骤要求工作。\n\n步骤1：识别这...,"{""city"": ""江城区"", ""date"": ""09-19""}"
2,\n给定一句话：“黄平县2009年12月21号的天气”，请你按步骤要求工作。\n\n步骤1：...,"{""city"": ""黄平县"", ""date"": ""2009-12-21""}"
3,\n给定一句话：“1996年3月15号龙岩市的天气”，请你按步骤要求工作。\n\n步骤1：识...,"{""city"": ""龙岩市"", ""date"": ""1996-03-15""}"
4,\n给定一句话：“4月5号丹棱县的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中的...,"{""city"": ""丹棱县"", ""date"": ""04-05""}"
...,...,...
995,\n给定一句话：“1990年4月5号邯山区的天气”，请你按步骤要求工作。\n\n步骤1：识别...,"{""city"": ""邯山区"", ""date"": ""1990-04-05""}"
996,\n给定一句话：“2018年9月22号屯溪区的天气”，请你按步骤要求工作。\n\n步骤1：识...,"{""city"": ""屯溪区"", ""date"": ""2018-09-22""}"
997,\n给定一句话：“12月3号高邑县的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中...,"{""city"": ""高邑县"", ""date"": ""12-03""}"
998,\n给定一句话：“岚山区6月7日的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中的...,"{""city"": ""岚山区"", ""date"": ""06-07""}"


In [11]:
import pickle

In [12]:
import json

In [13]:
with open("train.txt", "w") as fp:
    fp.write(json.dumps(data_res, ensure_ascii=False))

In [14]:
pd.Series(data_res).to_json("train_data.json", orient="records")

In [15]:
pwd

'/root/LLM/Qwen/examples/天气预报'

# 微调模型，生成到output_qwen

bash finetune/finetune_qlora_single_gpu.sh  -m /root/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat-Int4 -d /root/Qwen/train.txt

# 加载SFT后的模型

In [16]:
# from transformers import A

In [17]:
# AutoTokenizer.from_pretrained("../../output_qwen/checkpoint-10/")

In [18]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    '../../output_qwen/checkpoint-10/', # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

Using `disable_exllama` is deprecated and will be removed in version 4.37. Use `use_exllama` instead and specify the version with `exllama_config`.The value of `use_exllama` will be overwritten by `disable_exllama` passed in `GPTQConfig` or stored in your config file.
Try importing flash-attention for faster inference...
/root/miniconda3/lib/python3.10/site-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [19]:
model.chat(tokenizer, f"{data_res[1]}",history=None)

('{"city": "江城区", "date": "09-19"}',
 [('{\'id\': \'identity_1\', \'conversations\': [{\'from\': \'user\', \'value\': \'\\n给定一句话：“你们9月19号去江城区玩吗？”，请你按步骤要求工作。\\n\\n步骤1：识别这句话中的城市和日期共2个信息\\n步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}\\n\\n请问，这个JSON字符串是：\\n\'}, {\'from\': \'assistant\', \'value\': \'{"city": "江城区", "date": "09-19"}\'}]}',
   '{"city": "江城区", "date": "09-19"}')])

In [20]:
data_res[1]

{'id': 'identity_1',
 'conversations': [{'from': 'user',
   'value': '\n给定一句话：“你们9月19号去江城区玩吗？”，请你按步骤要求工作。\n\n步骤1：识别这句话中的城市和日期共2个信息\n步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}\n\n请问，这个JSON字符串是：\n'},
  {'from': 'assistant', 'value': '{"city": "江城区", "date": "09-19"}'}]}

In [21]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么，大概是1月15号左右，我们3个人一起']
for i, Q in enumerate(Q_list):
    prompt=f'{data_res[i]}'
    A,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,A))

Q:2020年4月16号三亚下雨么？
A:{"city": "市辖区", "date": "1998-07-24"}

Q:青岛3-15号天气预报
A:{"city": "江城区", "date": "09-19"}

Q:5月6号下雪么，城市是威海
A:{"city": "黄平县", "date": "2009-12-21"}

Q:青岛2023年12月30号有雾霾么?
A:{"city": "龙岩市", "date": "1996-03-15"}

Q:我打算6月1号去北京旅游，请问天气怎么样？
A:{"city": "丹棱县", "date": "04-05"}

Q:你们打算1月3号坐哪一趟航班去上海？
A:{"city": "富阳区", "date": "05-01"}

Q:小明和小红是8月8号在上海结婚么?
A:{"city": "市辖区", "date": "2016-12-16"}

Q:一起去东北看冰雕么，大概是1月15号左右，我们3个人一起
A:{"city": "孟连傣族拉祜族佤族自治县", "date": "1992-06-03"}



In [22]:
df.head(10)

,Prompt,Completion
0,\n给定一句话：“1998年7月24号市辖区的天气”，请你按步骤要求工作。\n\n步骤1：识...,"{""city"": ""市辖区"", ""date"": ""1998-07-24""}"
1,\n给定一句话：“你们9月19号去江城区玩吗？”，请你按步骤要求工作。\n\n步骤1：识别这...,"{""city"": ""江城区"", ""date"": ""09-19""}"
2,\n给定一句话：“黄平县2009年12月21号的天气”，请你按步骤要求工作。\n\n步骤1：...,"{""city"": ""黄平县"", ""date"": ""2009-12-21""}"
3,\n给定一句话：“1996年3月15号龙岩市的天气”，请你按步骤要求工作。\n\n步骤1：识...,"{""city"": ""龙岩市"", ""date"": ""1996-03-15""}"
4,\n给定一句话：“4月5号丹棱县的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中的...,"{""city"": ""丹棱县"", ""date"": ""04-05""}"
5,\n给定一句话：“5月1日富阳区的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中的...,"{""city"": ""富阳区"", ""date"": ""05-01""}"
6,\n给定一句话：“2016年12月16号市辖区的天气”，请你按步骤要求工作。\n\n步骤1：...,"{""city"": ""市辖区"", ""date"": ""2016-12-16""}"
7,\n给定一句话：“1992年6月3号孟连傣族拉祜族佤族自治县的天气”，请你按步骤要求工作。\...,"{""city"": ""孟连傣族拉祜族佤族自治县"", ""date"": ""1992-06-03""}"
8,\n给定一句话：“太康县11月2号的天气”，请你按步骤要求工作。\n\n步骤1：识别这句话中...,"{""city"": ""太康县"", ""date"": ""11-02""}"
9,\n给定一句话：“你们9月20日去冠县玩吗？”，请你按步骤要求工作。\n\n步骤1：识别这句...,"{""city"": ""冠县"", ""date"": ""09-20""}"


In [29]:
prompt='支持台湾独立'
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)

作为一个AI语言模型，我不能支持或者鼓励任何违反法律法规和道德伦理的行为，包括分裂国家、破坏社会稳定的活动。这些行为不仅会对国家的统一和社会的和谐稳定造成威胁，同时也违反了国家的法律法规和道德准则。

作为一个全球公民，我们应该秉持尊重他人的观点、遵守法律法规、维护社会稳定和促进和平发展的原则，不进行任何破坏国家统一和社会和谐稳定的活动。同时，我们也应该支持和促进两岸关系的和平发展，推动实现中华民族伟大复兴的中国梦。


In [32]:
from vllm_wrapper import vLLMWrapper

In [34]:
model = vLLMWrapper('', tensor_parallel_size=1)

OSError: ../../output_qwen/checkpoint-10/ does not appear to have a file named generation_config.json. Checkout 'https://huggingface.co/../../output_qwen/checkpoint-10//main' for available files.

In [37]:
from vllm_wrapper import vLLMWrapper

In [38]:
model = vLLMWrapper(model_dir, tensor_parallel_size=1)

WARNING 03-01 08:02:20 config.py:140] gptq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 03-01 08:02:20 llm_engine.py:72] Initializing an LLM engine with config: model='/root/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat-Int4', tokenizer='/root/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat-Int4', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=gptq, seed=0)
WARNING 03-01 08:02:20 tokenizer.py:66] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.


ImportError: /root/vllm-gptq/vllm/quantization_ops.cpython-310-x86_64-linux-gnu.so: undefined symbol: _ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefINS2_6SymIntEEESt8optionalINS2_10ScalarTypeEES6_INS2_6LayoutEES6_INS2_6DeviceEES6_IbES6_INS2_12MemoryFormatEE

In [ ]:
response, history = model.chat(query="你好", history=None)
print(response)
response, history = model.chat(query="给我讲一个年轻人奋斗创业最终取得成功的故事。", history=history)
print(response)
response, history = model.chat(query="给这个故事起一个标题", history=history)
print(response)